In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
#from tensorflow import keras
from tensorflow.keras import layers, models

! pip install tensorview
import tensorview as tv


import linora as la
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [4]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [5]:
IMG_WIDTH=200
IMG_HEIGHT=200
img_folder='data/'


In [6]:
def create_dataset(img_folder):
    file_count=0
    img_data_array=[]
    class_name=[]
    

    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
            #print(file)
            file_count+=1
            #print(file_count)
            image_path= os.path.join(img_folder, dir1,  file)
            image= cv2.imread( image_path, cv2.IMREAD_GRAYSCALE)
            image=np.array(image)
            image = image.astype('float32')
            image /= 255 
            image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
            img_data_array.append(image)
            if dir1=='covid':
                class_name.append(1)
            else:
                class_name.append(0)
            
    return img_data_array, class_name

Loading dataset

In [7]:
X, y = create_dataset(img_folder)


In [8]:
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (37, 200, 200)
37 train samples
19 test samples


In [16]:

# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same',input_shape=(200,200,1)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.BatchNormalization())
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.BatchNormalization())
# model.add(layers.Flatten())
# model.add(layers.Dense(512, activation='relu'))
# model.add(layers.Dense(512, activation='relu'))
# model.add(layers.Dense(2,activation='softmax'))

input_layer = tf.keras.Input(shape=(200, 200, 1))
x = tf.keras.layers.Conv2D(64, 3)(input_layer)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.MaxPool2D(2)(x)
x = tf.keras.layers.Conv2D(64, 3)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.Flatten()(x)
output_layer = tf.keras.layers.Dense(2, activation='softmax')(x)
model = tf.keras.Model(input_layer, output_layer)
model.compile(loss=['sparse_categorical_crossentropy'], 
                optimizer=tf.keras.optimizers.Adam(0.001),
                metrics=['accuracy'])


# opt = tf.keras.optimizers.Adam(learning_rate=0.0001,)
# model.compile(optimizer=opt,
#               loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#               metrics=['accuracy'])

# history = model.fit(train_data,train_labels, batch_size=4,epochs=100,validation_split=0.2)

In [ ]:
checkpoint = ModelCheckpoint("best_model.hdf5", monitor='loss', verbose=1,
    save_best_only=True, save_weights_only=True, mode='auto', save_freq=1)

In [ ]:
epochs = 10
batch_size = 16
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=[checkpoint])

In [ ]:
model.save_weights('weights.h5')

In [ ]:
import h5py
keys = []
final_keys = []
f = h5py.File("weights.h5",'r')

# with h5py.File("weights.h5",'r') as f: # open file
f.visit(keys.append) # append all keys to list
for key in keys:
    if ':' in key: # contains data if ':' in key
        print(f[key].name)
        final_keys.append(f[key].name)

/batch_normalization_2/batch_normalization_2/beta:0
/batch_normalization_2/batch_normalization_2/gamma:0
/batch_normalization_2/batch_normalization_2/moving_mean:0
/batch_normalization_2/batch_normalization_2/moving_variance:0
/batch_normalization_3/batch_normalization_3/beta:0
/batch_normalization_3/batch_normalization_3/gamma:0
/batch_normalization_3/batch_normalization_3/moving_mean:0
/batch_normalization_3/batch_normalization_3/moving_variance:0
/conv2d_2/conv2d_2/bias:0
/conv2d_2/conv2d_2/kernel:0
/conv2d_3/conv2d_3/bias:0
/conv2d_3/conv2d_3/kernel:0
/dense_1/dense_1/bias:0
/dense_1/dense_1/kernel:0


In [ ]:
group = f[key]

for i in final_keys:
    print(i)
    print(group[i][()].shape)
    print(group[i][()])

In [ ]:
weights_dict = {}

weight_callback = tf.keras.callbacks.LambdaCallback \
( on_epoch_end=lambda epoch, logs: weights_dict.update({epoch:model.get_weights()}))

history = model.fit( X_train, y_train, batch_size=8, epochs=5, callbacks=[weight_callback] )

# retrive weights
for epoch,weights in weights_dict.items():
    print("Weights for 2nd Layer of epoch #",epoch+1)
    print(weights[2])
    print("Bias for 2nd Layer of epoch #",epoch+1)
    print(weights[3])

In [ ]:
len(weights_dict[0][0])

In [10]:
import json

log_file = {}
weight_callback = tf.keras.callbacks.LambdaCallback \
( on_epoch_end=lambda epoch, logs: weights_dict.update({epoch:model.get_weights()}))

json_log = open('loss_log.json', mode='wt', buffering=1)
json_logging_callback = tf.keras.callbacks.LambdaCallback(
            on_epoch_end=lambda epoch, 
            # logs: json_log.write(
            #     json.dumps({'epoch': epoch, 
            #                 'loss': logs['loss'],
            #                 'weights': model.get_weights()}) + '\n'),
            logs: log_file.update({
                epoch: {
                    'epoch': epoch,
                    'loss' : logs['loss'],
                    'weights': model.get_weights()
                }
            }),
            on_train_end=lambda logs: json_log.close()
)

In [85]:
batch_size = 8
epochs = 5
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test),

          callbacks=[json_logging_callback])

Epoch 1/5
5/5 [==============================] - 2s 492ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.5881 - val_accuracy: 0.8421
Epoch 2/5
5/5 [==============================] - 2s 452ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 2.0156 - val_accuracy: 0.7368
Epoch 3/5
5/5 [==============================] - 2s 456ms/step - loss: 2.1568e-05 - accuracy: 1.0000 - val_loss: 4.2472 - val_accuracy: 0.7368
Epoch 4/5
5/5 [==============================] - 2s 468ms/step - loss: 1.2196 - accuracy: 0.9730 - val_loss: 1.1135 - val_accuracy: 0.8421
Epoch 5/5
5/5 [==============================] - 2s 441ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 6.8834 - val_accuracy: 0.4737


In [87]:
filter_weights = []
filter_weights_df = pd.DataFrame()
index = 0
for i in range(len(log_file[0]['weights'][0])):
    for j in range(len(log_file[0]['weights'][0][i])):
        filter_weights.append(log_file[0]['weights'][0][i][j][0])
        filter_weights_df[index] = log_file[0]['weights'][0][i][j][0]
        index += 1

In [89]:
filter_weights_df.to_csv('filter_weights.csv')

In [83]:
import pandas as pd
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
filter_weights = min_max_scaler.fit_transform(filter_weights)

In [84]:
filter_weights_df = pd.DataFrame(filter_weights.transpose())
filter_weights_df

,0,1,2,3,4,5,6,7,8
0,0.823784,0.534068,0.328726,0.835604,0.055848,0.120485,0.000000,0.100255,1.000000
1,0.457018,0.417920,0.162308,0.136650,1.000000,0.938431,0.000000,0.537630,0.834499
2,0.000000,0.459676,1.000000,0.202147,0.733210,0.920442,0.278218,0.580191,0.846649
3,0.699194,0.941569,0.168110,0.000000,0.075146,1.000000,0.101424,0.398990,0.148570
4,0.086521,0.466877,0.227804,0.334362,0.493192,0.234109,0.000000,0.562838,1.000000
...,...,...,...,...,...,...,...,...,...
59,0.604131,0.433610,0.000000,0.308819,0.242385,0.118685,0.663894,0.542264,1.000000
60,0.000000,0.577829,0.345768,1.000000,0.875258,0.336921,0.279481,0.305051,0.038248
61,0.000000,0.494332,0.280982,1.000000,0.636952,0.241384,0.247687,0.053938,0.972070
62,0.737442,0.150235,0.371387,0.764259,0.000000,0.359803,0.990279,0.605293,1.000000


In [81]:
import csv
w = csv.writer(open("output.csv", "w"))
for key, val in log_file.items():
    w.writerow([key, val])

In [ ]:
df = pd.DataFrame(our_data, columns=['loss', 'acc'])
df.to_csv('loss_acc.csv')

In [13]:
tv.model.statistics(model)

,layer_custom_name,layer_object_name,trainable,dtype,input_shape,output_shape,params
0,input_1,InputLayer,True,float32,"[(None, 200, 200, 1)]","[(None, 200, 200, 1)]",0
1,conv2d,Conv2D,True,float32,"(None, 200, 200, 1)","(None, 198, 198, 64)",640
2,batch_normalization,BatchNormalization,True,float32,"(None, 198, 198, 64)","(None, 198, 198, 64)",256
3,re_lu,ReLU,True,float32,"(None, 198, 198, 64)","(None, 198, 198, 64)",0
4,max_pooling2d,MaxPooling2D,True,float32,"(None, 198, 198, 64)","(None, 99, 99, 64)",0
5,conv2d_1,Conv2D,True,float32,"(None, 99, 99, 64)","(None, 97, 97, 64)",36928
6,batch_normalization_1,BatchNormalization,True,float32,"(None, 97, 97, 64)","(None, 97, 97, 64)",256
7,re_lu_1,ReLU,True,float32,"(None, 97, 97, 64)","(None, 97, 97, 64)",0
8,flatten,Flatten,True,float32,"(None, 97, 97, 64)","(None, 602176)",0
9,dense,Dense,True,float32,"(None, 602176)","(None, 2)",1204354


In [15]:
tv.model.visualize_weights(model, layer_name=['conv2d', 'batch_normalization', 'conv2d_1', 'batch_normalization_1' 'dense'])


'/Users/ashik/Documents/Courses/Spring 2021/Data visualization/Final project/visualize_weights.html'